## 모두를 위한 딥러닝 #2

Neural Network의 구성 요소와 학습에 도움을 주는 몇 가지 요소들에 대해서 소개한다. 

### Vanishing Gradient 문제 

Network가 깊어지면(*Deep*) Backpropagation을 했을 때 정보가 앞 쪽까지 전달되지 않는 ***Vanishing gradient*** 현상이 발생한다.<br>
sigmoid 함수로 예를 들어 설명해보자. network가 깊어진다는 의미는 activation function(*여기에선 sigmoid*)가 많이 쓰인다는 뜻이며 
backpropagation 시에 network를 지날 때마다 $\sigma(z)'$의 값이 계속 곱해질 것이다. 

아래 그래프에서 sigmoid의 값과 미분 값을 보면 불행하게도 가장 높았을 때의 미분 값이 0.25이다. <br> 
이 뜻은 backpropagation이 정보를 뒤로 전달해서 각 parameter들을 update하는 역할을 가지고 있는데 network가 깊어지면 1보다 작은 $\sigma(z)'$값이 계속 곱해지면서 0에 가까운 값으로 가버리게 된다. **즉, parameter의 update가 일어나지 않는다!**

![sigmoid and its derivative](https://cdn-images-1.medium.com/max/1600/1*Sek4P_MzBAipJJpwA8iS7Q.png)
<center>*&lt;sigmoid 함수 그래프, 미분 값 그래프&gt;*</center>


위에서 살펴봤을 때 문제는 바로 sigmoid의 미분값이 1보다 작았다는 점이다. 그럼, 1보다 크면? 무한대로 발산할 것이다. 이에 대한 해결책들을 아래에서 살펴보도록 하자.

#### 해결: Activation function
- **ReLU** <br>
현재 가장 대표적으로 많이 사용하는 activation function이다. ReLU function은 되게 간단하다. 음수 일 때는 0, 양수 일 때는 해당 값을 갖는다. backpropagation을 했을 때에도 미분 값이 1이나 0이기 때문에 vanishing gradient의 문제는 생기지 않고 다른 문제인 **dead ReLU**가 발생할 수 있다. gradient가 0이 될 수 있어서 node가 아예 죽어버리는 현상을 말하는 것인데 너무 많이 죽으면 결국 모델의 일정 부분이 쓰이지 않는 것이라고 볼 수 있어서 성능이 좋게 나오기 어렵다. <br><br>
ReLU로 잘 될까? 라는 의문이 들 수 있는데 non-linearity를 줄 수 있고(매우 복잡한 데이터를 표현해야 하는 딥러닝 모델에서 필수적) 실제로 학습시켰을 때 잘 작동한다.
![relu and sigmoid](https://cdn-images-1.medium.com/max/1600/1*XxxiA0jJvPrHEJHD4z893g.png)
<center>*&lt;ReLu vs sigmoid 비교&gt;*</center>

- **Leaky ReLU** <br>
위에서 발생한 문제를 기억해보면 값이 **음수일 때 gradient가 0으로 가는 것** 이 문제였다. 음..그럼 기울기를 조금만 주면 되지 않을까?
라는게 Leaky ReLU의 아이디어이다.

![relu and leaky relu](https://cdn-images-1.medium.com/max/1600/1*A_Bzn0CjUgOXtPCJKnKLqA.jpeg)
<center>*&lt;ReLu vs Leaky ReLu 비교&gt;*</center>


그 외에도 여러 가지 activation이 있는데 자세한 내용은 생략한다. (*sigmoid나 ReLU처럼 수식적으로 복잡하지 않아서 궁금하면 살펴보자.*)
- **maxout**
- **tanh**
- **ELU**



#### 해결: Weight initialization 
위 sigmoid 그래프를 다시 한 번 보자. 가중치를 학습 초기 단계에 지정을 해줘야 하는데 어떤 값이 적절한 값일까? 극단적으로 생각을 해보자.
- weight가 너무 작다: layer를 지날 때마다 값이 계속 작아진다. 그럼 sigmoid의 그래프를 보면 0 근처에서는 직선의 값을 갖기 때문에 network에 non-linearity를 줄 수 없어서 데이터에 맞게 모델의 모양이 학습이 잘 되지 않는다.
- weight가 너무 크다: layer를 지날 때마다 값이 계속 커진다. 그럼 sigmoid의 그래프를 봤을 때 backpropagation 시에 vanishing gradient 문제가 발생할 수 있다. <br><br>
결론은 weight가 너무 작지도 않고 크지도 않게, 초기화를 시키는 것이 중요하다. 대부분의 방법들이 0을 평균으로 하는 Gaussian 분포에서 표준편차를 조정해서 사용하며 가장 많이 사용하는 방법은 `Xavier Initialization`이다. 수식적인 내용이 궁금하면 아래 블로그를 보도록 하자. <br>
참고: http://www.khshim.com/archives/641

***
### Overfitting 문제

#### 해결: Regularization 
- **Dropout** <br>
아래 구조를 보자. dropout은 단순히 network내 node들의 연결을 random하게 끊어주는 것이다. 모델은 수많은 node들의 상호작용으로 학습 데이터에 대한 모델을 만든다. 직관적인 내용은 전문가들이 엄청 많을 때 몇 명 빼더라도 잘 맞출 것이며 한 번에 모든 전문가들이 평가하는 것보다 일부 전문가들만 평가를 반복하면서 학습을 하는 것이 조금 더 일반적인 모델을 학습시켜 overfitting을 막을 수 있다는 것이다. <br><br>
학습 시에 random하게 node를 제거한 뒤 학습하고 실제 평가할 때에는 전체 node를 다 쓴다
![dropout](http://cfile26.uf.tistory.com/image/2264014957A0248C2D18CD) <br><br>
<center>*&lt;dropout 구조 설명 &gt;*</center>
![dropout_explain](http://cfile26.uf.tistory.com/image/217D274F57A0248E2058AA) <br><br>
<center>*&lt;dropout이 왜 잘 작동하는 지에 대한 설명&gt;*</center>

***
### 학습 방법 
지난 Coursera 강의에서는 Gradient descent의 개념에 대해서만 살펴봤다. 마찬가지로 gradient descent에서 파생된 최적화 기법들이 아래와 같이 많은 종류가 있다. 가장 기본적인 방법이 SGD로, 샘플을 뽑은 뒤 단순히 gradient를 현재 parameter에 update 해준다. 
그리고 크게 2가지 직관이 추가가 되는데 첫째는 경사를 내려간다고 했을 때 앞선 방향을 고려해서 학습시켜주는 것이다. 그럴 경우 처음 가던 방향이 있기 때문에 좀 덜 흔들리게 목적지로 갈 수 있따. 두번째는 학습에 따른 가중치를 주는 것이다. 처음에는 갈 길이 머니까 빨리 학습하고 목적지를 거의 다 왔을 때는 천천히 가는 방법을 생각할 수 있다. 

위 직관들에 대한 최적화 방법들이 아래 내열된 방법들이며 다 알 필욘 없으며 가장 많이 사용하는 방법은 `Adam` 이다. 수식적인 내용이나 좀 더 깊은 이해가 필요하다면 http://cs231n.github.io/neural-networks-3/#sgd를 참조하자.

- SGD
- Momentum
- NAG
- Adagrad
- Adadelta
- Rmsprop
- Adamm

![optimizer](http://cs231n.github.io/assets/nn3/opt2.gif)

***
### 성능 개선

- **Batch Normalization** <br>
layer를 지날 때 내부 값들이 한 쪽으로 쏠리는 현상이 발생할 수 있다. 예를 들면 sigmoid function을 지날 때(위 그래프)에서 1에 가까운 값들이 
몰리게 되면 학습이 느려지게 된다. (vanishing gradient) 그래서, 이를 막기 위해서 layer의 값을 normalization 시켜주면 어떨까? 라는 접근 방법을 택한 것이 batch normalization 이다.

***
출처:
1. Sigmoid and its prime: https://cdn-images-1.medium.com/max/1600/1*Sek4P_MzBAipJJpwA8iS7Q.png
2. CS231n lecture slides
3. relu: https://cdn-images-1.medium.com/max/1600/1*XxxiA0jJvPrHEJHD4z893g.png
4. leaky relu vs relu: https://cdn-images-1.medium.com/max/1600/1*A_Bzn0CjUgOXtPCJKnKLqA.jpeg
5. CS231n lecture note: http://cs231n.github.io/assets/nn3/opt2.gif

